In [1]:
import os

import pandas as pd

In [2]:
data_path = 'dataset/demo/inputs/'

all_files = os.listdir(data_path)
games_inputs = {}
for filename in all_files:
    filepath = data_path + filename
    inputs = pd.read_csv(filepath)
    games_inputs[filename.split('.')[0]] = inputs[['joy_x', 'joy_y']] # we are only interested in joy_x and joy_y columns

games_inputs[all_files[5]].head(10)

,joy_x,joy_y
0,0.0000,0.0000
1,0.0000,0.0000
2,0.0000,0.0000
3,0.0000,0.0000
4,-0.9250,0.3625
5,-0.9250,0.3750
6,-0.9125,0.3875
7,-0.8875,0.4375
8,-0.8875,0.4500
9,-0.8875,0.4500


## Compute velocities

In [3]:
def nextDisplacement(df, index):
    velocities = [] # stores both x and y displacement
    for col in df.columns:
        velocities.append(df.at[index+1, col] - df.at[index, col])

    return velocities


In [4]:
game_displacements = {}
# games_inputs is a dictionary that contains input dataframes keyed by game 
for g_name in games_inputs.keys():
    game = games_inputs[g_name]
    
    ds = []
    prev_frame = 0
    for i in range(game.shape[0] - 1):
        # get next displacement
        d = nextDisplacement(game, i)

        if not d == [0, 0]: # only include frames that have some velocity
            
            # get elapsed
            elapsed = i - prev_frame
            prev_frame = i

            if elapsed > 60: # arbitrarily set all values > 60 to 0
                elapsed = 0

            ds.append((d[0], d[1], elapsed))

    game_displacements[g_name] = pd.DataFrame(ds, columns=['d_x', 'd_y', 'frames_elapsed'])

game_displacements[all_files[0]].head(10)

,d_x,d_y,frames_elapsed
0,-1.0000,0.0000,29
1,0.0125,0.0000,20
2,0.1125,-0.4625,28
3,0.2875,-0.3375,1
4,0.2250,-0.1250,1
5,0.0625,-0.0250,1
6,0.3000,0.0000,1
7,0.9875,0.9500,1
8,-0.0125,0.0000,3
9,0.0125,0.0000,1


## Normalize frames_elapsed
norm(x) = (x / max_x)

In [5]:
max = 60 # we know this is the maximum because we removed every value that was bigger

for game in game_displacements.values():
    game['frames_elapsed'] = game['frames_elapsed'].map(lambda x : x / max)

game_displacements[all_files[0]].head(10)

,d_x,d_y,frames_elapsed
0,-1.0000,0.0000,0.483333
1,0.0125,0.0000,0.333333
2,0.1125,-0.4625,0.466667
3,0.2875,-0.3375,0.016667
4,0.2250,-0.1250,0.016667
5,0.0625,-0.0250,0.016667
6,0.3000,0.0000,0.016667
7,0.9875,0.9500,0.016667
8,-0.0125,0.0000,0.050000
9,0.0125,0.0000,0.016667


## Write outputs to csv

In [6]:
output_dir = 'dataset/demo/displacements/'
for game in game_displacements.keys():
    game_displacements[game].to_csv(output_dir+game+'.csv', index=False)